In [128]:
import requests
import random
import time
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from lxml import etree

In [125]:
class Crawl_Liepin():

    user_agent = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
        'Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
        'Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)',
        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)',
    ]

    def get_main_url(self, page, city='sz'):
        url = 'https://www.liepin.com/city-%s/zhaopin/'%city
        if page >= 1:
            url += 'pn%s/'%(page-1)
        #url += '?key=&d_sfrom=search_city&d_ckId=c72351a8c506da0d2bf13032a46d9b98&d_curPage=1&d_pageSize=40&d_headId=c72351a8c506da0d2bf13032a46d9b98'
        return url

    # Crawl the page
    def crawl(self, url, sleep=0.1):
        headers = {'user-agent': random.choice(self.user_agent)}
        response = requests.get(url=url, headers=headers)
        time.sleep(sleep)
        return response.text

    def get_jd_url(self, text):
        f = etree.HTML(text)
        return [i for i in f.xpath('//@href') if '/job/' in i and 'shtml' in i]
    
    def parse_jd(self, text):
        dic = {}
        f = etree.HTML(text)
        job_url = self.retrieve(f.xpath('//link[@rel="canonical"]/@href'))
        dic['job_id'] = job_url.split('/')[-1][:-6]
        dic['job_title'] = self.retrieve(f.xpath('//h1/text()'))
        dic['job_url'] = job_url
        dic['company']= self.retrieve(f.xpath('//h3/a/@title'))
        dic['company_url']= self.retrieve(f.xpath('//h3/a/@href'))
        dic['salary']= self.retrieve(f.xpath("//p[@class='job-item-title']/text()")).strip()
        dic['location'] = self.retrieve(f.xpath("//p[@class='basic-infor']/span/a/text()"))
        if self.retrieve(f.xpath("//em[@class='job-title-icon-inclined']")) is not None:
            dic['eager'] = '急聘'
        dic['list_date'] = self.retrieve(f.xpath("//time[@title]/@title"))
        job_quali = f.xpath("//div[@class='job-qualifications']/span/text()")
        dic['degree'] = job_quali[0]
        dic['experience'] = job_quali[1]
        dic['language'] = job_quali[2]
        dic['age'] = job_quali[3]
        dic['tag'] = f.xpath("//ul[@class='comp-tag-list clearfix']/li/span/text()")
        dic['job_desc'] = [ii.strip() for ii in f.xpath("//div[@class='content content-word']/text()") if len(ii.strip())>0]
        dic['company_desc'] = [ii.strip() for ii in f.xpath("//div[@class='info-word']/text()") if len(ii.strip())>0]
        info_labels = f.xpath("//div[@class='content']/ul/li/span/text()")
        info_texts = f.xpath("//div[@class='content']/ul/li/label/text()")
        if len(info_labels) == len(info_texts):
            if "下属人数：" in info_labels:
                dic['num_of_subs'] = info_texts[info_labels.index("下属人数：")]
            if "所属部门：" in info_labels:
                dic['department'] = info_texts[info_labels.index("所属部门：")]
            if "汇报对象：" in info_labels:
                dic['supervisor'] = info_texts[info_labels.index("汇报对象：")]
            if "企业性质：" in info_labels:
                dic['type_of_company'] = info_texts[info_labels.index("企业性质：")]
            if "企业规模：" in info_labels:
                dic['scale_of_company'] = info_texts[info_labels.index("企业规模：")]
        return dic
    
    # Compatible with the xpath lib, return the first value if finded or None if not
    def retrieve(self, lis):
        if len(lis) > 0:
            return lis[0]
        else:
            return None

In [131]:
crawler = Crawl_Liepin()
job_urls = []
for i in tqdm(range(1, 101)):
    try:
        main_url = crawler.get_main_url(i)
        text = crawler.crawl(main_url)
        job_urls += crawler.get_jd_url(text)
    except: pass
result = []
for i in tqdm(job_urls):
    try:
        text = crawler.crawl(i)
        result.append(crawler.parse_jd(text))
    except: pass

In [132]:
df = pd.DataFrame(result)

In [137]:
df.to_excel('07-06.xlsx')

In [5]:
f = crawler.crawl(jd_urls[0])
f = etree.HTML(f)

In [72]:
crawler.retrieve(f.xpath("//p[@class='basic-infor']/span/a/text()"))

'深圳'

In [38]:
len(z.strip())

2

In [39]:
z.strip()

'面议'

In [92]:
f.xpath("//div[@class='content']/ul/li/span/text()")

['所属部门：', '专业要求：', '汇报对象：', '下属人数：']

In [93]:
f.xpath("//div[@class='content']/ul/li/label/text()")

['FAE', '不限', '无', '0人']